In [ ]:
import cv2
import numpy as np
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes=[]
with open ("coco.names",'r') as f:
    classes=[line.strip() for line in f.readlines()]
layers_names=net.getLayerNames()
outputlayers=[layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
img=cv2.imread("federer.jpg")
img=cv2.resize(img,None,fx=0.4,fy=0.4)
blob=cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)#separetes into different colors
height,width,channel=img.shape
net.setInput(blob)
outs=net.forward(outputlayers)
boxes=[]
confidences=[]
class_ids=[]
for out in outs:
    for detection in out:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence>0.4:
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            x=int(center_x-w/2)
            y=int(center_y-h/2)
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
#number_objects_detected=len(boxes)
indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
font=cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h=boxes[i]
        label=str(classes[class_ids[i]])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.putText(img,label,(x,y+30),font,3,(0,0,0),3)
    
    
cv2.imshow("",img)
cv2.waitKey(0)
cv2.destroyAllWindows()